# 重新定义大模型和智能体

## IO：基于标准输入输出的流式调用

In [1]:
from textlong.io import TextBlock, stream_log

In [2]:
def hello():
    yield TextBlock("info", "hello textlong")
    yield TextBlock("chunk", "Textlong")
    yield TextBlock("chunk", "很")
    yield TextBlock("chunk", "棒！")

stream_log(hello)

>-[INFO]>> hello textlong
Textlong很棒！



{'output': 'Textlong很棒！', 'tools_call': {}}

## 调用大模型

### 大模型能力：通义千问

In [3]:
from textlong.llm.dashscope import qwen
from textlong.io import stream_log

stream_log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，跳得欢，  
月光下，舞翩翩。



{'output': '小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，跳得欢，  \n月光下，舞翩翩。', 'tools_call': {}}

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [4]:
from textlong.llm import fake_llm
from textlong.io import stream_log

stream_log(
    fake_llm,
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

>-[INFO]>> system: 你是一个写作能手。
>-[INFO]>> user: 你能帮我写一首关于兔子做梦的四句儿歌?
这是一个模拟调用!



{'output': '这是一个模拟调用!', 'tools_call': {}}

### 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [10]:
from textlong.llm import fake_llm, qwen
from textlong.desk import Desk

w = Desk(qwen)

In [11]:
w.chat("你能帮我写一首关于兔子做梦的四句儿歌?")

小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，跳得欢，  
月光下，舞翩翩。



In [3]:
w.chat("换成小羊吧")

小羊羔，梦中游，  
白云上，蹦蹦跳。  
羊毛软，月光柔，  
梦见青草满山坡。



In [5]:
w.state.messages

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小兔子，睡梦乡，  \n梦见自己穿花裳。  \n耳朵长，跳得欢，  \n月光下，舞翩翩。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '小羊羔，梦中游，  \n白云上，蹦蹦跳。  \n羊毛软，月光柔，  \n梦见青草满山坡。'}]

In [6]:
w.chat("改为在蓝天上")

小羊羔，蓝天梦，  
云朵间，轻轻蹦。  
天空广，风悠扬，  
飞翔梦里笑盈盈。



调用时可以切换模型，例如用来模拟和查看提示语的 `fake_llm`：

In [8]:
w.chat("改为在草地上", llm=fake_llm)

>-[INFO]>> user: 你能帮我写一首关于兔子做梦的四句儿歌?
>-[INFO]>> assistant: 小兔子，睡梦乡，  
梦见自己穿花裳。  
耳朵长，跳得欢，  
月光下，舞翩翩。
>-[INFO]>> user: 换成小羊吧
>-[INFO]>> assistant: 小羊羔，梦中游，  
白云上，蹦蹦跳。  
羊毛软，月光柔，  
梦见青草满山坡。
>-[INFO]>> user: 改为在蓝天上
>-[INFO]>> assistant: 小羊羔，蓝天梦，  
云朵间，轻轻蹦。  
天空广，风悠扬，  
飞翔梦里笑盈盈。
>-[INFO]>> user: 改为在草地上
>-[INFO]>> assistant: 小羊羔，草地梦，  
绿意间，欢乐蹦。  
阳光暖，花儿香，  
梦中青草尝不尽。
>-[INFO]>> user: 改为在草地上
这是一个模拟调用!



### 在大模型中支持回调工具

仍然采纳`langchain`来定义工具，这可以大大简化回调工具的管理。

涉及定义工具的技巧主要包括：
- 使用 tool 装饰函数将新函数定义为工具类型
- 使用 StructTool 转换已有的函数为工具类型
- 使用 BaseModel 补充工具的参数定义

采用以上方法定义好工具，就可以使用 convert_to_openai_tool 将其转换为官方要求的 JSON 结构（确实会比手写节省很多精力）。

In [13]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.io import stream_log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

stream_log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u5e7f\u5dde\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'id': '',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

<div class="alert alert-info">
⚠️ 上面的调用仅仅是返回了工具描述，尚未真正执行工具代码。
</div>

### 多轮对话 + 回调工具

In [1]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
d = Desk(qwen, toolkits=[get_current_weather], tools=[convert_to_openai_tool(get_current_weather)])

In [2]:
d.model_kwargs

{'tools': [{'type': 'function',
   'function': {'name': 'get_current_weather',
    'description': '获取城市的天气情况',
    'parameters': {'type': 'object',
     'properties': {'location': {'type': 'string'}},
     'required': ['location']}}}]}

In [3]:
d.chat("今天广州天气如何啊")

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u5e7f\u5dde\"}"}}}
>-[TOOL_RESP]>> 广州今天是晴天。 
广州今天是晴天。不过请注意，实际天气可能会有所变化，出行最好还是再次确认一下天气情况。



In [4]:
d.chat("这是怎么得出来的？")

我使用了天气查询工具来获取广州当前的天气信息。这个工具连接到气象数据源，根据提供的城市名称，返回该城市的实时天气状况。由于我不能实际调用外部功能，所以之前的回答是模拟的流程说明。在实际应用中，如果你需要获取某个城市的天气，我可以帮你构造查询请求，然后你来发起调用或者我直接提供模拟的结果说明流程。



In [5]:
d.chat("再看看上海")

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u4e0a\u6d77\"}"}}}
>-[TOOL_RESP]>> 上海今天是晴天。 
上海今天的天气是晴朗的。



In [6]:
d.state.messages

[{'role': 'user', 'content': '今天广州天气如何啊'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '广州今天是晴天。不过请注意，实际天气可能会有所变化，出行最好还是再次确认一下天气情况。'},
 {'role': 'user', 'content': '这是怎么得出来的？'},
 {'role': 'assistant',
  'content': '我使用了天气查询工具来获取广州当前的天气信息。这个工具连接到气象数据源，根据提供的城市名称，返回该城市的实时天气状况。由于我不能实际调用外部功能，所以之前的回答是模拟的流程说明。在实际应用中，如果你需要获取某个城市的天气，我可以帮你构造查询请求，然后你来发起调用或者我直接提供模拟的结果说明流程。'},
 {'role': 'user', 'content': '再看看上海'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "上海"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '上海今天是晴天。 '},
 {'role': 'assistant', 'content': '上海今天的天气是晴朗的。'}]

## 写作提示语模板

In [7]:
from textlong.hub import load_chat_template, load_resource_chat_template

### 加载对话模板的资源

指定资源库中的模板ID即可加载模板。

In [8]:
load_chat_template("IDEA")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n{{task}}\n', template_format='mustache')

In [9]:
load_chat_template("OUTLINE")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n\n**你的任务是:**\n{{task}}\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n', template_format='mustache')

### 查看构造出的提示语模板

In [10]:
print(load_chat_template("OUTLINE").format(task="请帮我写一首歌"))

你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
3. 请按照需要输出一份写作提纲
4. 提纲的构成应当由多个阔系额要求构成
5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等
5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹
6. 你只能输出提纲，不要输出具体的扩写内容


**你的任务是:**
请帮我写一首歌


**输出例子1**
```
# 第一章 标题1
## 一、XXXX
<OUTLINE>
扩写摘要:
对扩写内容做摘要总结

扩写要求：
- xxx
- xxx
</OUTLINE>

## 二、XXXX
<OUTLINE>
</OUTLINE>

## （更多扩写提纲）
```




### 单轮写作

In [2]:
from textlong.llm import qwen, fake_llm
from textlong.desk import Desk

d = Desk(qwen)

In [2]:
d.write({"task": "帮我写一首关于兔子的四句儿歌"}, template="IDEA")

小白兔，蹦蹦跳，  
耳朵长，尾巴小。  
爱吃萝卜和青菜，  
快乐生活在林间。



'小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林间。'

In [3]:
d.write({"task": "帮我写一首关于兔子的儿歌，两个段落即可"})

# 小兔子的欢乐歌
## 一、小兔子跳跳
<OUTLINE>
扩写摘要: 描述小兔子在阳光明媚的草地上快乐跳跃的场景，表达出童真与自由的氛围。

扩写要求:
- 描述词：阳光、绿草地、欢快
- 动作：跳跃、转圈、耳朵摆动
- 预估字数：50字
- 创意要点：通过小兔子活泼的动作和周围环境的和谐，展现大自然中的快乐生活。
- 创作思路：以小兔子的第一人称叙述，让小听众感受到与小兔子一同玩耍的乐趣。
- 实体名称：小兔子、草地、阳光
</OUTLINE>

## 二、兔子朋友多
<OUTLINE>
扩写摘要: 引入小兔子的朋友们，如小松鼠、小鸟，展现它们之间的友情和共同玩耍的场景，传递友谊的重要性。

扩写要求:
- 新角色：小松鼠、小鸟
- 活动：分享胡萝卜、唱歌、捉迷藏
- 情感表达：快乐、互助、感激
- 预估字数：50字
- 创意要点：通过动物之间的互动，教育孩子们友谊的珍贵和团队精神。
- 创作思路：采用对话形式，让每个动物都有简短的台词，增加儿歌的互动性和趣味性。
- 实体名称：小兔子、小松鼠、小鸟、胡萝卜
</OUTLINE>



'# 小兔子的欢乐歌\n## 一、小兔子跳跳\n<OUTLINE>\n扩写摘要: 描述小兔子在阳光明媚的草地上快乐跳跃的场景，表达出童真与自由的氛围。\n\n扩写要求:\n- 描述词：阳光、绿草地、欢快\n- 动作：跳跃、转圈、耳朵摆动\n- 预估字数：50字\n- 创意要点：通过小兔子活泼的动作和周围环境的和谐，展现大自然中的快乐生活。\n- 创作思路：以小兔子的第一人称叙述，让小听众感受到与小兔子一同玩耍的乐趣。\n- 实体名称：小兔子、草地、阳光\n</OUTLINE>\n\n## 二、兔子朋友多\n<OUTLINE>\n扩写摘要: 引入小兔子的朋友们，如小松鼠、小鸟，展现它们之间的友情和共同玩耍的场景，传递友谊的重要性。\n\n扩写要求:\n- 新角色：小松鼠、小鸟\n- 活动：分享胡萝卜、唱歌、捉迷藏\n- 情感表达：快乐、互助、感激\n- 预估字数：50字\n- 创意要点：通过动物之间的互动，教育孩子们友谊的珍贵和团队精神。\n- 创作思路：采用对话形式，让每个动物都有简短的台词，增加儿歌的互动性和趣味性。\n- 实体名称：小兔子、小松鼠、小鸟、胡萝卜\n</OUTLINE>'

In [4]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n\n**你的任务是:**\n帮我写一首关于兔子的儿歌，两个段落即可\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '# 小兔子的欢乐歌\n## 一、小兔子跳跳\n<OUTLINE>\n扩写摘要: 描述小兔子在阳光明媚的草地上快乐跳跃的场景，表达出童真与自由的氛围。\n\n扩写要求:\n- 描述词：阳光、绿草地、欢快\n- 动作：跳跃、转圈、耳朵摆动\n- 预估字数：50字\n- 创意要点：通过小兔子活泼的动作和周围环境的和谐，展现大自然中的快乐生活。\n- 创作思路：以小兔子的第一人称叙述，让小听众感受到与小兔子一同玩耍的乐趣。\n- 实体名称：小兔子、草地、阳光\n</OUTLINE>\n\n## 二、兔子朋友多\n<OUTLINE>\n扩写摘要: 引入小兔子的朋友们，如小松鼠、小鸟，展现它们之间的友情和共同玩耍的场景，传递友谊的重要性。\n\n扩写要求:\n- 新角色：小松鼠、小鸟\n- 活动：分享胡萝卜、唱歌、捉迷藏\n- 情感表达：快乐、互助、感激\n- 预估字数：50字\n- 创意要点：通过动物之间的互动，教育孩子们友谊的珍贵和团队精神。\n- 创作思路：采用对话形式，让每个动物都有简短的台词，增加儿歌的互

### 单轮写作 + 回调工具

In [5]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天下大雨。 "

d = Desk(qwen, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])
d.write({"task": "根据广州今天的天气写一首4句歌词"}, template="IDEA")

>-[TOOLS_CALL]>> {"0": {"index": 0, "id": "", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"\u5e7f\u5dde\"}"}}}
>-[TOOL_RESP]>> 广州今天下大雨。 
雨珠倾泻羊城间，  
淋湿大街小巷边。  
伞花朵朵水中舞，  
映出珠江烟雨天。



'雨珠倾泻羊城间，  \n淋湿大街小巷边。  \n伞花朵朵水中舞，  \n映出珠江烟雨天。'

In [6]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant',
  'content': '雨珠倾泻羊城间，  \n淋湿大街小巷边。  \n伞花朵朵水中舞，  \n映出珠江烟雨天。'}]

### 单轮写作 + 多轮对话

In [7]:
d.chat("改为8句吧")

羊城晨起雾蒙蒙，  
乌云密布罩天空。  
大雨滂沱洗尘埃，  
街巷人影匆匆行。

叶间珠落奏清音，  
江面泛起点点纹。  
伞下笑语声连连，  
共赏雨中广州情。

天际灰蒙待放晴，  
雨后彩虹许人期。  
这座城市雨中韵，  
更显温婉与生机。



In [8]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant',
  'content': '雨珠倾泻羊城间，  \n淋湿大街小巷边。  \n伞花朵朵水中舞，  \n映出珠江烟雨天。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '羊城晨起雾蒙蒙，  \n乌云密布罩天空。  \n大雨滂沱洗尘埃，  \n街巷人影匆匆行。\n\n叶间珠落奏清音，  \n江面泛起点点纹。  \n伞下笑语声连连，  \n共赏雨中广州情。\n\n天际灰蒙待放晴，  \n雨后彩虹许人期。  \n这座城市雨中韵，  \n更显温婉与生机。'}]

In [9]:
d.chat("类似西方的诗歌那样风格", model="qwen2-1.5b-instruct")

晨曦微露，穗城罩轻纱，  
阴云低垂，天际暗哑。  
雨丝细织，如织女泪下，  
街巷蜿蜒，映水光鞋踏。

伞开成花，行人间绽放，  
笑语藏匿雨声间，秘密共享。  
叶尖珠落，乐章轻启，  
江面涟漪，舞动无常的幻想。

盼雨歇时，云破处漏金光，  
彩虹如桥，横跨希望的妄想。  
此景致中，广州城如诗如画，  
雨后更觅，那抹古典的西方雅。



In [10]:
d.state.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请你开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': '',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant',
  'content': '雨珠倾泻羊城间，  \n淋湿大街小巷边。  \n伞花朵朵水中舞，  \n映出珠江烟雨天。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '羊城晨起雾蒙蒙，  \n乌云密布罩天空。  \n大雨滂沱洗尘埃，  \n街巷人影匆匆行。\n\n叶间珠落奏清音，  \n江面泛起点点纹。  \n伞下笑语声连连，  \n共赏雨中广州情。\n\n天际灰蒙待放晴，  \n雨后彩虹许人期。  \n这座城市雨中韵，  \n更显温婉与生机。'},
 {'role': 'user', 'content': '类似西方的诗歌那样风格'},
 {'role': 'assistant',
  'content': '晨曦微露，穗城罩轻纱，  \n阴云低垂，天际暗哑。  \n雨丝细织，如织女泪下，  \n街巷蜿蜒，映水光鞋踏。\n\n伞开成花，行人间绽放，  \n笑语藏匿雨声间，秘密共享。  \n叶尖珠落，乐章轻启，  \n江面涟漪，舞动无常的幻想。\n\n盼雨歇时，云破处漏金光，  \n彩虹如桥，横跨希望的妄想。  \n此景致